# Rating Products

In [ ]:
Bir ürüne verilen puanlar üzerinden çeşitli değerlendirmeler yaparak en doğru puanın nasıl hesaplanabileceğine dair bir uygulama yapacağız.

In [1]:
# - Average
# - Time Based Weighted Average
# - User-Based Weighted Average
# - Weighted Rating

In [1]:
###################################
# AVERAGE
###################################

In [3]:
##########################################
# Uygulama: Kullanıcı ve Zaman Ağırlıklı Kurs Puanı Hesaplama
##########################################

In [5]:
import pandas as pd
import math
import scipy.stats as st
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [5]:
#Örnek Kurs: 
#(50+ Saat) Python A-Z: Veri Bilimi ve Machine Learning
#Toplam Puan: 4611
#Puan Yüzdeleri: 75, 20, 4, 1, <1
#Yaklaşık Sayısal Karşılıkları: 3458, 922, 184, 46, 6

df = pd.read_csv("C:/Users/emre.kanarya/Desktop/course_reviews.csv")
df.head()

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered
0,5.00000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.00000,0.00000,0.00000
1,5.00000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.00000,0.00000,0.00000
2,4.50000,2021-02-04 20:34:03,2019-07-04 23:23:27,1.00000,0.00000,0.00000
3,5.00000,2021-02-04 16:56:28,2021-02-04 14:41:29,10.00000,0.00000,0.00000
4,4.00000,2021-02-04 15:00:24,2020-10-13 03:10:07,10.00000,0.00000,0.00000


In [6]:
df.shape # Toplam kaç tane değerlendirme var buna bakalım.

(4323, 6)

In [21]:
#Puanların dağılımına bakalım:
df["Rating"].value_counts()   

5.00000    3267
4.50000     475
4.00000     383
3.50000      96
3.00000      62
1.00000      15
2.00000      12
2.50000      11
1.50000       2
Name: Rating, dtype: int64

In [22]:
#Sorulan sayısının dağılımına bakalım:
df["Questions Asked"].value_counts()

0.00000     3867
1.00000      276
2.00000       80
3.00000       43
4.00000       15
5.00000       13
6.00000        9
8.00000        5
9.00000        3
14.00000       2
11.00000       2
7.00000        2
10.00000       2
15.00000       2
22.00000       1
12.00000       1
Name: Questions Asked, dtype: int64

In [14]:
#Sorulan soru kırılımında verilen puan. Örneğin yukarıda 2 soru soran 80 kişinin ortalama kaç puan verdiğini merak ediyorum.
df.groupby("Questions Asked").agg({"Questions Asked": "count",
                                   "Rating": "mean"})

,Questions Asked,Rating
Questions Asked,,
0.00000,3867,4.76519
1.00000,276,4.74094
2.00000,80,4.80625
3.00000,43,4.74419
4.00000,15,4.83333
5.00000,13,4.65385
6.00000,9,5.00000
7.00000,2,4.75000
8.00000,5,4.90000


In [15]:
#Bizim ana amacımız bu kursa verilen puanların puanını hesaplamak.

######################
# Average
######################

# Ortalama Puan
df["Rating"].mean()

4.764284061993986

Fakat bu yukarıda bulunan örneğin ilk 3 ay iydir puanı son 3 ay ivme düşmeye başlamıştır. Bu yüzden yukarıda bulunan net puanı vermez bizim zamana göre puan ortalamasını almamız gerekiyor. Kurs yenilenmiş olur ya da market ise yeni ürünler gelmiş olabilir son zamanlarda gibi. Bu neden ile zamana göre olursa daha isabetli puanlanmış olur.

In [5]:
#######################################################################
# Puan Zamanlarına Göre Ağırlıklı Ortalama(Time Based Weighted Average)
#######################################################################

In [25]:
df.head()

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered
0,5.00000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.00000,0.00000,0.00000
1,5.00000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.00000,0.00000,0.00000
2,4.50000,2021-02-04 20:34:03,2019-07-04 23:23:27,1.00000,0.00000,0.00000
3,5.00000,2021-02-04 16:56:28,2021-02-04 14:41:29,10.00000,0.00000,0.00000
4,4.00000,2021-02-04 15:00:24,2020-10-13 03:10:07,10.00000,0.00000,0.00000


In [9]:
# Dikkat edilirse yukarıda "timestamp" değişkeni "object" bizim bunu "zaman" değişkenine çevirmemiz gerekmektedir.
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4323 entries, 0 to 4322
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Rating              4323 non-null   float64       
 1   Timestamp           4323 non-null   datetime64[ns]
 2   Enrolled            4323 non-null   object        
 3   Progress            4323 non-null   float64       
 4   Questions Asked     4323 non-null   float64       
 5   Questions Answered  4323 non-null   float64       
dtypes: datetime64[ns](1), float64(4), object(1)
memory usage: 202.8+ KB


In [11]:
# Şimdi yorumları "gün" cinsinden ifade etmemiz gerekiyor. Puanların hangi tarihte verildiği, yorumların hangi tarihte yapıldığı bilgisi var bu "timestamp" değişkeni.
# Dolaysıyla öyle bir işlem yapmamız lazım ki "bugünün tarihi" diye bir tarih belirleyelim ve bu tarihtem yorumların yapıldığı tarihi çıkaralım. 
current_date = pd.to_datetime('2021-02-10 0:0:0')
df["days"] = (current_date - df["Timestamp"]).dt.days

In [29]:
df.head()

,Rating,Timestamp,Enrolled,Progress,Questions Asked,Questions Answered,days
0,5.00000,2021-02-05 07:45:55,2021-01-25 15:12:08,5.00000,0.00000,0.00000,4
1,5.00000,2021-02-04 21:05:32,2021-02-04 20:43:40,1.00000,0.00000,0.00000,5
2,4.50000,2021-02-04 20:34:03,2019-07-04 23:23:27,1.00000,0.00000,0.00000,5
3,5.00000,2021-02-04 16:56:28,2021-02-04 14:41:29,10.00000,0.00000,0.00000,5
4,4.00000,2021-02-04 15:00:24,2020-10-13 03:10:07,10.00000,0.00000,0.00000,5


In [20]:
# Şimdi bu veri seti içerisinde son 30 günde yapılan yorumlara nasıl erişirim?
df[df["days"] <= 30].count()

Rating                194
Timestamp             194
Enrolled              194
Progress              194
Questions Asked       194
Questions Answered    194
days                  194
dtype: int64

In [21]:
# Son 30 günde yapılan yorumların ortalamasını almak istiyoruz.
df.loc[df["days"]<=30, "Rating"].mean()

4.775773195876289

In [32]:
# 30 dan büyük 90 dan küçük eşit olanları bir çağıralım ve bunlarında ortalamasını alalım.

df.loc[(df["days"] > 30) & (df["days"] <= 90), "Rating"].mean()

4.763833992094861

In [33]:
df.loc[(df["days"] > 90) & (df["days"] <= 180), "Rating"].mean()

4.752503576537912

In [34]:
df.loc[(df["days"] > 180), "Rating"].mean()

4.76641586867305

In [22]:
#Şimdi bu işlemi kod ile otomatikleştirelim, yani yüzdeliğe vuralım.
df.loc[df["days"]<=30, "Rating"].mean() * 28/100 + \
df.loc[(df["days"] > 30) & (df["days"] <= 90), "Rating"].mean() * 26/100 + \
df.loc[(df["days"] > 90) & (df["days"] <= 180), "Rating"].mean() * 24/100 + \
df.loc[(df["days"] > 180), "Rating"].mean() * 22/100

4.765025682267194

Yukarıda görüldüğü üzre "zamana göre ortalama puan" ı hesapladık.

In [12]:
#Fonksiyonlaştıralım:
def time_based_weighted_average(dataframe, w1=28, w2=26, w3=24, w4=22):
    return dataframe.loc[df["days"]<=30, "Rating"].mean() * w1/100 + \
           dataframe.loc[(dataframe["days"] > 30) & (dataframe["days"] <= 90), "Rating"].mean() * w2/100 + \
           dataframe.loc[(dataframe["days"] > 90) & (dataframe["days"] <= 180), "Rating"].mean() * w3/100 + \
           dataframe.loc[(dataframe["days"] > 180), "Rating"].mean() * w4/100
time_based_weighted_average(df)

4.765025682267194

Peki şöyle bir soru sorulabilir: "Herkesin verdiği puanlar aynı öneme mi sahip?" Bu soruyu aşağıda çözelim.

In [35]:
#################################################################
# Kullanıcı Temelli Ağırlıklı Ortalama(User-Based Weighted Average)
#################################################################

In [13]:
#Acaba bütün kullanıcılarınm verdiği puanlar aynı ağırlığa mı sahip olmalı? Bu soruya cevap arıyorum şuan.
#Örneğin kursun tamamını izleyen biri ile, kursun yüzde birini izleyen birisinin puanı aynı mı olacak?
#İzlenme oranlarına göre ağırlıklı puanlar buna göre mi ayrılmalı bu soruyu soruyorum.
df.groupby("Progress").agg({"Rating": "mean"})


,Rating
Progress,
0.00000,4.67391
1.00000,4.64269
2.00000,4.65476
3.00000,4.66355
4.00000,4.77733
5.00000,4.69821
6.00000,4.75510
7.00000,4.73256
8.00000,4.74194


In [14]:
df.loc[df["Progress"]<=10, "Rating"].mean() * 22/100 + \
df.loc[(df["Progress"] > 10) & (df["Progress"] <= 45), "Rating"].mean() * 24/100 + \
df.loc[(df["Progress"] > 45) & (df["Progress"] <= 75), "Rating"].mean() * 26/100 + \
df.loc[(df["Progress"] > 75), "Rating"].mean() * 28/100

4.800257704672543

In [15]:
#Fonksiyonlaştıralım:
def user_based_weighted_average(dataframe, w1=22, w2=24, w3=26, w4=28):
    return dataframe.loc[df["Progress"]<=30, "Rating"].mean() * w1/100 + \
           dataframe.loc[(dataframe["Progress"] > 10) & (dataframe["Progress"] <= 45), "Rating"].mean() * w2/100 + \
           dataframe.loc[(dataframe["Progress"] > 45) & (dataframe["Progress"] <= 75), "Rating"].mean() * w3/100 + \
           dataframe.loc[(dataframe["Progress"] > 75), "Rating"].mean() * w4/100
user_based_weighted_average(df, 20, 24, 26, 30)

4.808450632723469

Peki days ve progress e göre ortalamasını ayrı ayrı aldım ama bunların ikisni bir arada görsem daha iyi olmaz mıydı? Aşağıda bunu yapalım.

In [ ]:
##################################################
# Ağırlıklı Derecelendirme(Weighted Rating)
##################################################

Burada yaptığımız tüm işlemleri bir araya getiriyor olacağız. Tek bir fonksiyon kullanarak hesaplama işlemi gerçekleştireceğiz.

In [16]:
#Bir fonksiyon tanımlayalım:
def course_weighted_rating(dataframe, time_w=50, user_w=50):
    return time_based_weighted_average(dataframe) * time_w/100 + user_based_weighted_average(dataframe)*user_w/100
course_weighted_rating(df)

4.785481983483173

In [17]:
course_weighted_rating(df, time_w=40, user_w=60)

4.78957324372637

# Sorting Products

In [18]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [19]:
df = pd.read_csv('C:/Users/emre.kanarya/Desktop/product_sorting.csv')
print(df.shape)
df.head(10)

(32, 10)


,course_name,instructor_name,purchase_count,rating,commment_count,5_point,4_point,3_point,2_point,1_point
0,(50+ Saat) Python A-Z™: Veri Bilimi ve Machine...,Veri Bilimi Okulu,17380,4.80000,4621,3466,924,185,46,6
1,Python: Yapay Zeka ve Veri Bilimi için Python ...,Veri Bilimi Okulu,48291,4.60000,4488,2962,1122,314,45,45
2,5 Saatte Veri Bilimci Olun (Valla Billa),Instructor_1,18693,4.40000,2362,1582,567,165,24,24
3,R ile Veri Bilimi ve Machine Learning (35 Saat),Veri Bilimi Okulu,6626,4.60000,1027,688,257,51,10,21
4,(2020) Python ile Makine Öğrenmesi (Machine Le...,Veri Bilimi Okulu,11314,4.60000,969,717,194,38,10,10
5,Course_1,Instructor_2,4601,4.80000,213,164,45,4,0,0
6,Course_2,Instructor_3,3171,4.70000,856,582,205,51,9,9
7,Veri Bilimi için İstatistik: Python ile İstati...,Veri Bilimi Okulu,929,4.50000,126,88,26,9,0,3
8,A'dan Z'ye Apache Spark (Scala & Python),Veri Bilimi Okulu,6920,4.70000,214,154,41,13,2,4
9,Modern R Programlama Eğitimi,Veri Bilimi Okulu,6537,4.40000,901,559,252,72,9,9


In [4]:
# Rating e göre sıralayalım. Rating in objektif bir şekilde sıralandığını düşünüyoruz diğer bölüm de nasıl sıralandığını görmüştük.

In [5]:
#####################
# Sorting by Rating
#####################

In [20]:
df.sort_values("rating", ascending=False).head(20)
#Dikkat edilirse 4.satırda bulunan  "Course_1" isimli kurs yüksek puanlı fakat bu kursun yorum sayısı diğerlerine kıyasla oldukça düşük.
#Bunun dışında daha aşağıda bulunan kurslarda daha fazla satın alma sayısı olan kurslar var fakat rating in altında ezilmiş yani puanı yüksek olarak sıraladığımızda kursları
#2 faktör gözden kaçmış gibi görünüyor "satın alma sayıları", "yorum sayıları" gözden kaçmış görünüyor.
#Üçüncü problem ise yorum ve satın alma değişkenini ayrı ayrı düşünebiliriz, birlikte zaten ifade ettik. Satın alma sayısı(purchase_count) ve yorum sayısını(comment_count) göz ardı ettik. Bu ikisi aslında bu kararlara vardıracak, kullanıcılara satın almalarını sağlayacak bir kavram vardı bu sosyal ispatdı.
#Bu sosyal ispatı etkileyen en önemli etmenler bunlar. Evet puanı iyi fakat 5 puan almış fakat 3 kişi yorumlamış ile 4.5 puan almış 100 kişi yorumlamış bunlar bir değil. Sadece ratinge göre sıralama yapmak mantıklı değil. Öyle bir şey yapmalıyız ki hem satın alma sayısını hem puanı hem de yorum sayısını aynı anda göz önünde bulunduralım.

,course_name,instructor_name,purchase_count,rating,commment_count,5_point,4_point,3_point,2_point,1_point
0,(50+ Saat) Python A-Z™: Veri Bilimi ve Machine...,Veri Bilimi Okulu,17380,4.80000,4621,3466,924,185,46,6
10,İleri Düzey Excel|Dashboard|Excel İp Uçları,Veri Bilimi Okulu,9554,4.80000,2266,1654,499,91,22,0
19,Alıştırmalarla SQL Öğreniyorum,Veri Bilimi Okulu,3155,4.80000,235,200,31,4,0,0
5,Course_1,Instructor_2,4601,4.80000,213,164,45,4,0,0
6,Course_2,Instructor_3,3171,4.70000,856,582,205,51,9,9
14,Uçtan Uca SQL Server Eğitimi,Veri Bilimi Okulu,12893,4.70000,2425,1722,510,145,24,24
8,A'dan Z'ye Apache Spark (Scala & Python),Veri Bilimi Okulu,6920,4.70000,214,154,41,13,2,4
13,Course_5,Instructor_6,6056,4.70000,144,82,46,12,1,3
27,Course_15,Instructor_1,1164,4.60000,98,65,24,6,0,3
1,Python: Yapay Zeka ve Veri Bilimi için Python ...,Veri Bilimi Okulu,48291,4.60000,4488,2962,1122,314,45,45


In [32]:
#Şimdi sıralama işlemini hem yorum sayılarına hem de satın alma sayısına göre sıralayalım:
df.sort_values("purchase_count", ascending=False).head(20) # Satın alma sayısına göre sıraladık.

,course_name,instructor_name,purchase_count,rating,commment_count,5_point,4_point,3_point,2_point,1_point
1,Python: Yapay Zeka ve Veri Bilimi için Python ...,Veri Bilimi Okulu,48291,4.60000,4488,2962,1122,314,45,45
11,Course_3,Instructor_4,24809,4.30000,250,95,87,51,12,5
2,5 Saatte Veri Bilimci Olun (Valla Billa),Instructor_1,18693,4.40000,2362,1582,567,165,24,24
0,(50+ Saat) Python A-Z™: Veri Bilimi ve Machine...,Veri Bilimi Okulu,17380,4.80000,4621,3466,924,185,46,6
20,Course_9,Instructor_3,12946,4.50000,3371,2191,877,203,33,67
14,Uçtan Uca SQL Server Eğitimi,Veri Bilimi Okulu,12893,4.70000,2425,1722,510,145,24,24
15,Uygulamalarla SQL Öğreniyorum,Veri Bilimi Okulu,11397,4.50000,2353,1435,705,165,24,24
4,(2020) Python ile Makine Öğrenmesi (Machine Le...,Veri Bilimi Okulu,11314,4.60000,969,717,194,38,10,10
10,İleri Düzey Excel|Dashboard|Excel İp Uçları,Veri Bilimi Okulu,9554,4.80000,2266,1654,499,91,22,0
8,A'dan Z'ye Apache Spark (Scala & Python),Veri Bilimi Okulu,6920,4.70000,214,154,41,13,2,4


In [33]:
df.sort_values("commment_count", ascending=False).head(20) # Yorum sayısına göre sıraladık.

,course_name,instructor_name,purchase_count,rating,commment_count,5_point,4_point,3_point,2_point,1_point
0,(50+ Saat) Python A-Z™: Veri Bilimi ve Machine...,Veri Bilimi Okulu,17380,4.80000,4621,3466,924,185,46,6
1,Python: Yapay Zeka ve Veri Bilimi için Python ...,Veri Bilimi Okulu,48291,4.60000,4488,2962,1122,314,45,45
20,Course_9,Instructor_3,12946,4.50000,3371,2191,877,203,33,67
14,Uçtan Uca SQL Server Eğitimi,Veri Bilimi Okulu,12893,4.70000,2425,1722,510,145,24,24
2,5 Saatte Veri Bilimci Olun (Valla Billa),Instructor_1,18693,4.40000,2362,1582,567,165,24,24
15,Uygulamalarla SQL Öğreniyorum,Veri Bilimi Okulu,11397,4.50000,2353,1435,705,165,24,24
10,İleri Düzey Excel|Dashboard|Excel İp Uçları,Veri Bilimi Okulu,9554,4.80000,2266,1654,499,91,22,0
3,R ile Veri Bilimi ve Machine Learning (35 Saat),Veri Bilimi Okulu,6626,4.60000,1027,688,257,51,10,21
4,(2020) Python ile Makine Öğrenmesi (Machine Le...,Veri Bilimi Okulu,11314,4.60000,969,717,194,38,10,10
9,Modern R Programlama Eğitimi,Veri Bilimi Okulu,6537,4.40000,901,559,252,72,9,9


In [21]:
#Şimdi bu üç işlemi(rating, comment, purchase) rating, yorum ve satın alma sayısına göre sıralayalım.
#İlk önce standartlaştırma işlemini yazalım buraya:
#from sklearn.preprocessing import MinMaxScaler # Bunu ekleyelim kütüphanemize
import pandas as pd
from sklearn.preprocessing import MinMaxScaler # Bu metodu standartlaştırma işlemi için kullanacağız.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [ ]:
################################
# Sorting by Rating, Comment and Purchase
################################

In [22]:
# Şimdi yapacak olduğumuz şey bu üç faktörü birlikte aynı anda göz önünde bulundurmaya çalışmak. Nasıl yapabiliriz?
# Bu üç değişkeni aynı ölçeğe getirelim. Mesela rating in cinsinden ifade edelim. Rating 1-5 arasında sayılardan oluşuyor.
# Dolaysıyla biz de diğer değişkenleri 1 ile 5 arasındaki sayılara dönüştürelim. Bu işlemi hemen gerçekleştirelim.
df["purchase_count_scaled"] = MinMaxScaler(feature_range=(1, 5)). \
fit(df[["purchase_count"]]). \
transform(df[["purchase_count"]])

In [37]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
purchase_count,32.00000,7110.71875,9760.89340,30.00000,877.50000,3687.50000,9994.00000,48291.00000
rating,32.00000,4.35625,0.44789,3.10000,4.27500,4.50000,4.62500,4.80000
commment_count,32.00000,882.06250,1321.49890,7.00000,87.75000,194.50000,983.50000,4621.00000
5_point,32.00000,598.09375,920.14011,1.00000,49.25000,112.50000,695.25000,3466.00000
4_point,32.00000,211.53125,312.26291,2.00000,19.75000,45.50000,253.25000,1122.00000
3_point,32.00000,54.12500,76.57623,0.00000,6.75000,14.50000,56.25000,314.00000
2_point,32.00000,9.53125,12.99127,0.00000,0.75000,3.00000,10.50000,46.00000
1_point,32.00000,8.96875,14.57015,0.00000,2.00000,3.00000,9.00000,67.00000
purchase_count_scaled,32.00000,1.58687,0.80901,1.00000,1.07024,1.30314,1.82584,5.00000


In [23]:
#Yukarıda by-ulunan işlemin benzerini yorum için yapalım:
df["comment_count_scaled"] = MinMaxScaler(feature_range=(1, 5)). \
fit(df[["commment_count"]]). \
transform(df[["commment_count"]])

In [40]:
df.head()

,course_name,instructor_name,purchase_count,rating,commment_count,5_point,4_point,3_point,2_point,1_point,purchase_count_scaled,comment_count_scaled
0,(50+ Saat) Python A-Z™: Veri Bilimi ve Machine...,Veri Bilimi Okulu,17380,4.80000,4621,3466,924,185,46,6,2.43801,5.00000
1,Python: Yapay Zeka ve Veri Bilimi için Python ...,Veri Bilimi Okulu,48291,4.60000,4488,2962,1122,314,45,45,5.00000,4.88470
2,5 Saatte Veri Bilimci Olun (Valla Billa),Instructor_1,18693,4.40000,2362,1582,567,165,24,24,2.54684,3.04161
3,R ile Veri Bilimi ve Machine Learning (35 Saat),Veri Bilimi Okulu,6626,4.60000,1027,688,257,51,10,21,1.54669,1.88427
4,(2020) Python ile Makine Öğrenmesi (Machine Le...,Veri Bilimi Okulu,11314,4.60000,969,717,194,38,10,10,1.93525,1.83398


In [41]:
#Şimdi artık üçü de aynı cinsten bunların ortalamasını alabilirz. Yüzdelik yapıp sıralayalım:
(df["comment_count_scaled"] * 32 / 100 +
 df["purchase_count_scaled"] * 26 / 100 +
 df["rating"] * 42 / 100)

0    4.24988
1    4.79510
2    3.48349
3    2.93711
4    3.02204
5    2.75165
6    2.85721
7    2.52239
8    2.75990
9    2.81623
10   3.42792
11   2.98739
12   2.52869
13   2.72186
14   3.50198
15   3.36577
16   2.51798
17   2.28232
18   2.45807
19   2.72659
20   3.68156
21   2.51906
22   2.53828
23   2.35459
24   2.26479
25   2.02105
26   2.43612
27   2.56168
28   2.54467
29   1.92584
30   1.92400
31   2.27376
dtype: float64

In [24]:
#Yukarıda bulunan çıktı bizim SKOR larımızdır. Birçok faktörün ağırlığı ile oluşturulmuştur.
#Peki bu ağırlıkları değiştirmek istiyorum bunun için bir fonksiyon yazalım:
def weighted_sorting_score(dataframe, w1=32, w2=26, w3=42):
    return(dataframe["comment_count_scaled"] * w1 / 100 +
           dataframe["purchase_count_scaled"] * w1 / 100 +
           dataframe["rating"] * w1 / 100)

In [25]:
# Şimdi fonksiyonumuzu çağırıp dataframe içerisine "weighted_sorting_score" adında yeni bir değişken oluşturabiliriz.
df["weighted_sorting_score"] = weighted_sorting_score(df)


In [26]:
df.sort_values("weighted_sorting_score", ascending=False).head(20)
#Artık görüldüğü üzere yukarıda ilk üçte olanlar bu sıralamada yok çünkü rating, satın alma ve yoruma göre sıraladık. Bu bizim için daha güvenilir oldu. 

,course_name,instructor_name,purchase_count,rating,commment_count,5_point,4_point,3_point,2_point,1_point,purchase_count_scaled,comment_count_scaled,weighted_sorting_score
1,Python: Yapay Zeka ve Veri Bilimi için Python ...,Veri Bilimi Okulu,48291,4.60000,4488,2962,1122,314,45,45,5.00000,4.88470,4.63510
0,(50+ Saat) Python A-Z™: Veri Bilimi ve Machine...,Veri Bilimi Okulu,17380,4.80000,4621,3466,924,185,46,6,2.43801,5.00000,3.91616
20,Course_9,Instructor_3,12946,4.50000,3371,2191,877,203,33,67,2.07051,3.91634,3.35579
2,5 Saatte Veri Bilimci Olun (Valla Billa),Instructor_1,18693,4.40000,2362,1582,567,165,24,24,2.54684,3.04161,3.19630
14,Uçtan Uca SQL Server Eğitimi,Veri Bilimi Okulu,12893,4.70000,2425,1722,510,145,24,24,2.06612,3.09623,3.15595
10,İleri Düzey Excel|Dashboard|Excel İp Uçları,Veri Bilimi Okulu,9554,4.80000,2266,1654,499,91,22,0,1.78937,2.95839,3.05528
15,Uygulamalarla SQL Öğreniyorum,Veri Bilimi Okulu,11397,4.50000,2353,1435,705,165,24,24,1.94213,3.03381,3.03230
11,Course_3,Instructor_4,24809,4.30000,250,95,87,51,12,5,3.05375,1.21066,2.74061
4,(2020) Python ile Makine Öğrenmesi (Machine Le...,Veri Bilimi Okulu,11314,4.60000,969,717,194,38,10,10,1.93525,1.83398,2.67815
3,R ile Veri Bilimi ve Machine Learning (35 Saat),Veri Bilimi Okulu,6626,4.60000,1027,688,257,51,10,21,1.54669,1.88427,2.56991


In [27]:
# Burada ilgisiz olan yani veri bilimi keyword u ile aratıyoruz excel vs. geliyor bu alakasız gelenleri çıkarmak istiyoruz:
df[df["course_name"].str.contains("Veri Bilimi")].sort_values("weighted_sorting_score", ascending=False).head(20)

,course_name,instructor_name,purchase_count,rating,commment_count,5_point,4_point,3_point,2_point,1_point,purchase_count_scaled,comment_count_scaled,weighted_sorting_score
1,Python: Yapay Zeka ve Veri Bilimi için Python ...,Veri Bilimi Okulu,48291,4.60000,4488,2962,1122,314,45,45,5.00000,4.88470,4.63510
0,(50+ Saat) Python A-Z™: Veri Bilimi ve Machine...,Veri Bilimi Okulu,17380,4.80000,4621,3466,924,185,46,6,2.43801,5.00000,3.91616
3,R ile Veri Bilimi ve Machine Learning (35 Saat),Veri Bilimi Okulu,6626,4.60000,1027,688,257,51,10,21,1.54669,1.88427,2.56991
7,Veri Bilimi için İstatistik: Python ile İstati...,Veri Bilimi Okulu,929,4.50000,126,88,26,9,0,3,1.07451,1.10316,2.13686


# Bayes Ortalama Derecelendirme Puanı

Problemimiz hala sıralama problemidir. Bu konu karşımıza "beş yıldızlı sistemlerde ürün sıralama" ya da "Beş yıldızın dağılımına göre ürün sıralama" olarak karşımıza çıkabilir. 

In [49]:
df.head()

,course_name,instructor_name,purchase_count,rating,commment_count,5_point,4_point,3_point,2_point,1_point,purchase_count_scaled,comment_count_scaled,weighted_sorting_score
0,(50+ Saat) Python A-Z™: Veri Bilimi ve Machine...,Veri Bilimi Okulu,17380,4.80000,4621,3466,924,185,46,6,2.43801,5.00000,3.91616
1,Python: Yapay Zeka ve Veri Bilimi için Python ...,Veri Bilimi Okulu,48291,4.60000,4488,2962,1122,314,45,45,5.00000,4.88470,4.63510
2,5 Saatte Veri Bilimci Olun (Valla Billa),Instructor_1,18693,4.40000,2362,1582,567,165,24,24,2.54684,3.04161,3.19630
3,R ile Veri Bilimi ve Machine Learning (35 Saat),Veri Bilimi Okulu,6626,4.60000,1027,688,257,51,10,21,1.54669,1.88427,2.56991
4,(2020) Python ile Makine Öğrenmesi (Machine Le...,Veri Bilimi Okulu,11314,4.60000,969,717,194,38,10,10,1.93525,1.83398,2.67815


Amacımız yani bayes ne işe yarar? DF içerisinde yer alan puan dağılım sütunlarının(5 point, 4 point....) üzerinden ağırlıklı bir şekilde olasılıksal ortalama hesabı yapar. Yani bir ortalama hesaplayacağız. Bu puan sütunlarının üzerinden bir ortalama hesaplayacağız.
Sıralama kaygımız sürüyor hala, bu sıralama üzerindeki faktörlerden olan "rating" ile ilgili yeni bir yöntem öğreniyoruz. "Bayes den average rating" yöntemi.
Bu yöntem ne yapıyor? Her bir kurs için ayrı ayrı olan bu puanların(5 point sütunu vs.) dağılım bilgisini kullanarak bize bir average rating ortalama puanı yapacak. Daha sonra istresek buna göre sıralayabiliriz o zaman farklı problemle bizi bekliyor olacak ya da istersek hibrit bir yaklaşım gerçekleştiririz.

In [ ]:
Evet şimdi bu fonksiyon içinde kullanacağımız bazı fonksiyonlar, metotlar var.
"import math" ve 
"import scipy.stats as st" Biz bunları çalışmanın başında import etmiştik. Burada tekrar etmeyeceğiz.

In [28]:
def bayesian_average_rating(n, confidence=0.95):
    if sum(n) ==0:
        return 0
    K =len(n)
    z=st.norm.ppf(1-(1-confidence) / 2)
    N = sum(n)
    first_part = 0.0
    second_part=0.0
    for k, n_k in enumerate(n):
        first_part += (k+1) * (n[k]+1) / (N + K)
        second_part +=(k+1) * (k+1) * (n[k] + 1) / (N +K)
    score = first_part - z * math.sqrt((second_part - first_part * first_part) / (N + K + 1))
    return score
    

Şimdi yeni bir problemimiz var. Bu hesaplama işlemini nasıl gerçekleştireceğiz? Öyle bir işlem yapmalıyım ki veri seti içerisinde sadece puan sütunlarını seçeyeim(5_point vs.) daha sonra bunların hepsini "n" argümanına göndereyim ama ters bir sırada yani 5 4 değil de 1 2 şeklinde. "Confidince" ise hesaplanacak olan z tablo değerine ilişkin bir değer elde edebilmek adına girilmiş bir değerdir.  

In [29]:
# Şimdi adım adım gidelim:
df["bar_score"] = df.apply(lambda x: bayesian_average_rating(x[["1_point",
                                                                "2_point",
                                                                "3_point",
                                                                "4_point",
                                                                "5_point"]]) , axis=1)

In [61]:
# Bizim daha önce yaptığımız listeleme yani yorum, rating ve satın almayı ekleyerek ve ağrılık vererek bir skor oluşturmuştuk. Onu bir getirelim:
df.sort_values("weighted_sorting_score", ascending=False).head(20)

,course_name,instructor_name,purchase_count,rating,commment_count,5_point,4_point,3_point,2_point,1_point,purchase_count_scaled,comment_count_scaled,weighted_sorting_score,bar_score
1,Python: Yapay Zeka ve Veri Bilimi için Python ...,Veri Bilimi Okulu,48291,4.60000,4488,2962,1122,314,45,45,5.00000,4.88470,4.63510,4.51604
0,(50+ Saat) Python A-Z™: Veri Bilimi ve Machine...,Veri Bilimi Okulu,17380,4.80000,4621,3466,924,185,46,6,2.43801,5.00000,3.91616,4.66586
20,Course_9,Instructor_3,12946,4.50000,3371,2191,877,203,33,67,2.07051,3.91634,3.35579,4.48063
2,5 Saatte Veri Bilimci Olun (Valla Billa),Instructor_1,18693,4.40000,2362,1582,567,165,24,24,2.54684,3.04161,3.19630,4.51521
14,Uçtan Uca SQL Server Eğitimi,Veri Bilimi Okulu,12893,4.70000,2425,1722,510,145,24,24,2.06612,3.09623,3.15595,4.56816
10,İleri Düzey Excel|Dashboard|Excel İp Uçları,Veri Bilimi Okulu,9554,4.80000,2266,1654,499,91,22,0,1.78937,2.95839,3.05528,4.64168
15,Uygulamalarla SQL Öğreniyorum,Veri Bilimi Okulu,11397,4.50000,2353,1435,705,165,24,24,1.94213,3.03381,3.03230,4.45481
11,Course_3,Instructor_4,24809,4.30000,250,95,87,51,12,5,3.05375,1.21066,2.74061,3.87774
4,(2020) Python ile Makine Öğrenmesi (Machine Le...,Veri Bilimi Okulu,11314,4.60000,969,717,194,38,10,10,1.93525,1.83398,2.67815,4.59567
3,R ile Veri Bilimi ve Machine Learning (35 Saat),Veri Bilimi Okulu,6626,4.60000,1027,688,257,51,10,21,1.54669,1.88427,2.56991,4.48208


In [62]:
# Şimdi yeni oluşturduğumuz bar_score a göre bir sıralama yapalım:
df.sort_values("bar_score", ascending= False). head(20) # Bu sıralama dikkat edilirse satın alma ve yorumu es geçerek sadece puana odaklanılan bir yöntem yani ilk sırada bakılorsa düşük yıldız sayısı 0 olduğu için en yükseğe onu koymuş bu sadece puan tabanlı bir yöntem olsaydı evet doığru bir yöntemdi faka diğer kriterlerde bize lazım.

,course_name,instructor_name,purchase_count,rating,commment_count,5_point,4_point,3_point,2_point,1_point,purchase_count_scaled,comment_count_scaled,weighted_sorting_score,bar_score
19,Alıştırmalarla SQL Öğreniyorum,Veri Bilimi Okulu,3155,4.80000,235,200,31,4,0,0,1.25901,1.19766,2.32213,4.72913
0,(50+ Saat) Python A-Z™: Veri Bilimi ve Machine...,Veri Bilimi Okulu,17380,4.80000,4621,3466,924,185,46,6,2.43801,5.00000,3.91616,4.66586
10,İleri Düzey Excel|Dashboard|Excel İp Uçları,Veri Bilimi Okulu,9554,4.80000,2266,1654,499,91,22,0,1.78937,2.95839,3.05528,4.64168
5,Course_1,Instructor_2,4601,4.80000,213,164,45,4,0,0,1.37886,1.17859,2.35438,4.63448
4,(2020) Python ile Makine Öğrenmesi (Machine Le...,Veri Bilimi Okulu,11314,4.60000,969,717,194,38,10,10,1.93525,1.83398,2.67815,4.59567
14,Uçtan Uca SQL Server Eğitimi,Veri Bilimi Okulu,12893,4.70000,2425,1722,510,145,24,24,2.06612,3.09623,3.15595,4.56816
1,Python: Yapay Zeka ve Veri Bilimi için Python ...,Veri Bilimi Okulu,48291,4.60000,4488,2962,1122,314,45,45,5.00000,4.88470,4.63510,4.51604
2,5 Saatte Veri Bilimci Olun (Valla Billa),Instructor_1,18693,4.40000,2362,1582,567,165,24,24,2.54684,3.04161,3.19630,4.51521
6,Course_2,Instructor_3,3171,4.70000,856,582,205,51,9,9,1.26033,1.73602,2.46283,4.50797
3,R ile Veri Bilimi ve Machine Learning (35 Saat),Veri Bilimi Okulu,6626,4.60000,1027,688,257,51,10,21,1.54669,1.88427,2.56991,4.48208


In [30]:
#Şimdi daha yakından odaklanalım:
df[df["course_name"].index.isin([5, 1])].sort_values("bar_score", ascending=False)
#Evet bakıldığında 1.indexte bulunan daha fazla yorum ve satın almaya sahipken bundan yukarı çıktı çünkü bar_score yukarıda geldi.
#Bunun sebebi 5.indexte bulunan çok daha az sayıda puana sahip olduğu halde düşük puan miktarları diğer kursa göre daha az olmasından dolayı bu kurs yukarı çıkmıştır. Puanı evet 5.indexte bulunanın daha yüksek olabilir ama belirleyici ana etken bu değildir.
#Bu yöntem sadece rating lerin dağılımına baktığından dolayı,  sadece puana odaklandığından dolayı daha yüksek puanalra sahip olan dağılım açısından çok daha yüksek. 
#Evet bizim ne yapmamız gerekiyor diğer faktörleride değerlenirmemiz lazım bunu aşağıda yapacağız.

,course_name,instructor_name,purchase_count,rating,commment_count,5_point,4_point,3_point,2_point,1_point,purchase_count_scaled,comment_count_scaled,weighted_sorting_score,bar_score
5,Course_1,Instructor_2,4601,4.80000,213,164,45,4,0,0,1.37886,1.17859,2.35438,4.63448
1,Python: Yapay Zeka ve Veri Bilimi için Python ...,Veri Bilimi Okulu,48291,4.60000,4488,2962,1122,314,45,45,5.00000,4.88470,4.63510,4.51604


# Karma Sıralama(Hybrid Sorting)

In [37]:
def hybrid_sorting_score(dataframe, bar_w=60, wss_w=40):
    bar_score = dataframe.apply(lambda x: bayesian_average_rating(x[["1_point",
                                                                     "2_point",
                                                                     "3_point",
                                                                     "4_point",
                                                                     "5_point"]]), axis=1)
    
    wss_score = weighted_sorting_score(dataframe)
    return bar_score*bar_w/100 + wss_score*wss_w/100

In [40]:
#eri setimiz içine yerleştirelim:
df["hybrid_sorting_score"] = hybrid_sorting_score(df)
df.sort_values("hybrid_sorting_score", ascending= False).head(20)

,course_name,instructor_name,purchase_count,rating,commment_count,5_point,4_point,3_point,2_point,1_point,purchase_count_scaled,comment_count_scaled,weighted_sorting_score,bar_score,hybrid_sorting_score
1,Python: Yapay Zeka ve Veri Bilimi için Python ...,Veri Bilimi Okulu,48291,4.60000,4488,2962,1122,314,45,45,5.00000,4.88470,4.63510,4.51604,4.56366
0,(50+ Saat) Python A-Z™: Veri Bilimi ve Machine...,Veri Bilimi Okulu,17380,4.80000,4621,3466,924,185,46,6,2.43801,5.00000,3.91616,4.66586,4.36598
20,Course_9,Instructor_3,12946,4.50000,3371,2191,877,203,33,67,2.07051,3.91634,3.35579,4.48063,4.03069
10,İleri Düzey Excel|Dashboard|Excel İp Uçları,Veri Bilimi Okulu,9554,4.80000,2266,1654,499,91,22,0,1.78937,2.95839,3.05528,4.64168,4.00712
14,Uçtan Uca SQL Server Eğitimi,Veri Bilimi Okulu,12893,4.70000,2425,1722,510,145,24,24,2.06612,3.09623,3.15595,4.56816,4.00328
2,5 Saatte Veri Bilimci Olun (Valla Billa),Instructor_1,18693,4.40000,2362,1582,567,165,24,24,2.54684,3.04161,3.19630,4.51521,3.98765
15,Uygulamalarla SQL Öğreniyorum,Veri Bilimi Okulu,11397,4.50000,2353,1435,705,165,24,24,1.94213,3.03381,3.03230,4.45481,3.88581
4,(2020) Python ile Makine Öğrenmesi (Machine Le...,Veri Bilimi Okulu,11314,4.60000,969,717,194,38,10,10,1.93525,1.83398,2.67815,4.59567,3.82867
19,Alıştırmalarla SQL Öğreniyorum,Veri Bilimi Okulu,3155,4.80000,235,200,31,4,0,0,1.25901,1.19766,2.32213,4.72913,3.76633
5,Course_1,Instructor_2,4601,4.80000,213,164,45,4,0,0,1.37886,1.17859,2.35438,4.63448,3.72244


In [41]:
df[df["course_name"].str.contains("Veri Bilimi")].sort_values("hybrid_sorting_score", ascending=False).head(20)

,course_name,instructor_name,purchase_count,rating,commment_count,5_point,4_point,3_point,2_point,1_point,purchase_count_scaled,comment_count_scaled,weighted_sorting_score,bar_score,hybrid_sorting_score
1,Python: Yapay Zeka ve Veri Bilimi için Python ...,Veri Bilimi Okulu,48291,4.60000,4488,2962,1122,314,45,45,5.00000,4.88470,4.63510,4.51604,4.56366
0,(50+ Saat) Python A-Z™: Veri Bilimi ve Machine...,Veri Bilimi Okulu,17380,4.80000,4621,3466,924,185,46,6,2.43801,5.00000,3.91616,4.66586,4.36598
3,R ile Veri Bilimi ve Machine Learning (35 Saat),Veri Bilimi Okulu,6626,4.60000,1027,688,257,51,10,21,1.54669,1.88427,2.56991,4.48208,3.71721
7,Veri Bilimi için İstatistik: Python ile İstati...,Veri Bilimi Okulu,929,4.50000,126,88,26,9,0,3,1.07451,1.10316,2.13686,4.34219,3.46006


# IMDB Film Puanlama ve Sıralama(IMDB Movie Scoring and Sorting)

In [1]:
import pandas as pd
import math
pd.set_option("display.max_columns", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.float_format", lambda x: '%.5f' % x)

In [4]:
df = pd.read_csv("C:/Users/emre.kanarya/Desktop/movies_metadata.csv", low_memory=False)

In [9]:
df = df[["title", "vote_average", "vote_count"]] # Bizim işçin gerekli 3 değişken var.

In [7]:
df.head()

,title,vote_average,vote_count
0,Toy Story,7.70000,5415.00000
1,Jumanji,6.90000,2413.00000
2,Grumpier Old Men,6.50000,92.00000
3,Waiting to Exhale,6.10000,34.00000
4,Father of the Bride Part II,5.70000,173.00000


In [8]:
df.shape

(45466, 3)

Problem nedir?
IMDB de işe girdik ve bizden birçok kişinin film izlemek için referans aldığı bu listeyi güncelle denildi.
İlk 250 yi gerçekleştirmek hedefinde sıralamayı yapmamız beklenmektedir.

In [11]:
# Diyelim ki burada bulunan vote_average a göre bir sıralama yapmak istiyoruz:

##############################
# Vote Average'a Göre Sıralama
##############################
df.sort_values("vote_average", ascending=False).head(20) #Görüldüğü üzere sadece puan ortalamasına göre sıralamak mantıklı değil çünkü beklenilen filmlerin dışında filmler geldi. Vote_Count değerine bakılırsa sadece bir puan almış en üst sırada bulunan film.

,title,vote_average,vote_count
21642,Ice Age Columbus: Who Were the First Americans?,10.00000,1.00000
15710,If God Is Willing and da Creek Don't Rise,10.00000,1.00000
22396,Meat the Truth,10.00000,1.00000
22395,Marvin Hamlisch: What He Did For Love,10.00000,1.00000
35343,Elaine Stritch: At Liberty,10.00000,1.00000
186,Reckless,10.00000,1.00000
45047,The Human Surge,10.00000,1.00000
22377,The Guide,10.00000,1.00000
22346,هیچ کجا هیچ کس,10.00000,1.00000
1634,Other Voices Other Rooms,10.00000,1.00000


In [12]:
# O zaman şöyle bir varsayımda bulunalım: Örneğin vote_count u şundan büyük olana göre önce bir filtre koy sonra ortalamaya göre sırala diyelim.
# Peki iyi de o zaman vote_count kaç yapacağım? Bunun için describe atacağım:
df["vote_count"].describe([0.10, 0.25, 0.50, 0.70, 0.80, 0.90, 0.95, 0.99]).T

count   45460.00000
mean      109.89734
std       491.31037
min         0.00000
10%         1.00000
25%         3.00000
50%        10.00000
70%        25.00000
80%        50.00000
90%       160.00000
95%       434.00000
99%      2183.82000
max     14075.00000
Name: vote_count, dtype: float64

In [14]:
#Evet oylara bakıldığında medyan değeri 10 olarak görünüyor. Oy ortalamasında baktığımızda 109.89 olarak görünüyor. Yani filmlerin ortalama oy sayısı 100 civarındaymış. %90 ve %95 üzerinde olan 400 ün üzerinde. Biz top 250 dediğimiz için 400 den büyük olması bizim için mantıklı olacaktır.
df[df["vote_count"]>400].sort_values("vote_average", ascending=False).head(20) # Oy sayısı 400 den büyük olanlara göre sıralayacak.

,title,vote_average,vote_count
10309,Dilwale Dulhania Le Jayenge,9.10000,661.00000
40251,Your Name.,8.50000,1030.00000
834,The Godfather,8.50000,6024.00000
314,The Shawshank Redemption,8.50000,8358.00000
1152,One Flew Over the Cuckoo's Nest,8.30000,3001.00000
1176,Psycho,8.30000,2405.00000
1178,The Godfather: Part II,8.30000,3418.00000
292,Pulp Fiction,8.30000,8670.00000
1184,Once Upon a Time in America,8.30000,1104.00000
5481,Spirited Away,8.30000,3968.00000


In [17]:
# Görüldüğü üzere sadece "vote_count" a göre sıralama yaptığımızda acaba oy sayısını daha mı yukarı çekmeliydim gibi sorularım var. Neden sadece vote_count a göre sıralama yaptık. Şöyle bir şey daha mantıklı olabilirdi:
# vote_count u 1-10 arasına çekip çünkü vote_average 1-10 arasında çünkü. Bundan sonra bunları çarpıp sıralamak daha mantıklı olacaktır. Bu değerleri birlikte tek sadece bir değişkene göre değil de birlikte kullanırsak daha iyi sonuç alabileceğiz gibi gözüküyor.
# Bunun için metodumuzu getirelim aşağıda:
from sklearn.preprocessing import MinMaxScaler # Yapmak istediğimiz şey burada bulunan vote_count değişkenini standartlaştırmak.
df["vote_count_score"] = MinMaxScaler(feature_range=(1, 10)).fit(df[["vote_count"]]).transform(df[["vote_count"]])

In [22]:
#Böylelikle vote_average ve vote_count u aynı tip haline getirdik şimdi çarpıp sonuca bakabiliriz:
df.head() # vote_count_score ile vote_average değişkenlerini çarpacağız.   İkiside 1-10 arası değerler artık.
df["average_count_score"] = df["vote_average"] * df["vote_count_score"]

In [30]:
# Şimdi bu oluşturduğum değişkene göre veri setimizi sıralayalım:
df.sort_values("average_count_score", ascending=False).head()

,title,vote_average,vote_count,vote_count_score,average_count_score
15480,Inception,8.10000,14075.00000,10.00000,81.00000
12481,The Dark Knight,8.30000,12269.00000,8.84519,73.41505
22879,Interstellar,8.10000,11187.00000,8.15332,66.04190
17818,The Avengers,7.40000,12000.00000,8.67318,64.18153
14551,Avatar,7.20000,12114.00000,8.74607,62.97174


# IMDB Ağırlıklı Derecelendirme ( IMDB Weighted Rating )

In [ ]:
2015 yılına kadar IMDB nin kullandığı skor. Yani IMDB de çalışan ilgili geliştiriciler oturmuş ve biz bütün dünyayı ilgilendiren
bu sıralamayı nasıl yapalım diye düşündüğünde, karar kıldıkları yöntemi değerlendireceğiz. Filmleri bu yönteme göre sıralamışlar.
IMDB ner ki ben ağırlıklı bir rating hesaplayacağım, bunu hesaplarken göz önünde bulunduracağım iki durum var:
Yöntemin formülü:
WR = (v/v+m*R) + (m/v+m*c)
- C: Kitlenin genel ortalaması, tüm filmlerin genel ortalaması.
- m: Sıralamaya girebilmek için gerekli minimum oy sayısı
- R: Her bir filmin kendi puanını ifade eder.    
- v: İlgili filmin oy sayısıdır.

In [ ]:
# ÖRNEK:
#Film 1:
#r=8 (Filmin ortalama puanı)
#M=500 (Gereken oy sayısı)
#v=1000 (Filmin aldığı oy sayısı) 

In [44]:
#Bu film için formülasyonun sol tarafını hesaplıyorum. Sağ tarafa henüz geçmedik.
#(v/v+m*R)
a1=(1000/(1000 + 500)) * 8
#####################################################################
#Formülasyonun sağ tarafını da hesaplayalım:
#(m/v+m*c)
a2=500/(1000+500) * 7
a1+a2

7.666666666666666

In [45]:
#Diyelim ki başka bir film olsun. Bu film aynı puana sahip olsun fakat daha fazla OY almış olsun.
#Film 2:
#r=8
#M=500
#v=3000
#Bu aynı puana sahip olan filmin puanını nasıl etkiliyor olacak?
#(v/(v+m)) * R
b1=(3000/(3000+500)) * 8
################################################################################
#Formülasyonun sağ tarafını da hesaplayalım:
#(m/v+m*c)
b2=500/(3000+500) * 7
b1+b2

7.857142857142857

Çıktı da görüldüğü üzere daha fazla oy alan bir film matematiksel olarak daha fazla puan almış oluyor. Bu durum matematiksel olarak yakalanmış oldu.

In [38]:
#Puanı 9.5 olan bir filmi inceleyip bakalım:
(1000 / (1000+500)) * 9.5

6.333333333333333

Görüldüğü üzere puanı 8 olan filmin yorum sayısı fazla olduğu için bunu geçemedi 9.5 puana sahip olan bu film.

# SONUÇ:
Bir işyeri bir problemde kendisine has bir skorlama yöntemi, kendisine has bir ağırlıklı ortalama hesaplama yöntemi bulabilir.
Nuradaki temel problem "ortalama hesabı". Bizim amacımızne ?
Tamam bu basit bir ortalama hesabı fakat bunu hepsi için yaptığım zaman, problem bir sıralama problemidir. Anlaşıldığı üzere burada bütün kitlenin ortalaması ve minimum gereken oy sayısı,
her bir filmin kendi içindeki oy sayısı ve kendi puanına göre bir işleme tabi tutulup ortalama bir ağırlık hesabı yapılıyor.

In [119]:
#Yukarıda bulunan sonucu matematiksel olarak ifade edecek olursak:
#M = 2500 (Minimum gereken oy sayısı)   
#C = df["vote_average"].mean() #Kitlenin ortalaması veri seti içinde bulunan ortalama puan baz alınarak hesaplandı.
# Bu işlemi bir fonksiyon ile tanımlayacak olursak:
def weighted_rating(r, v, M, C): # Her bir film için bu hesaplama yapılması gerekiyor.
    return(v / (v + M) * r) + (M / (v + M) * C)

In [59]:
#Fonksiyonumu kayıt ettim. Şimdi her bir film için bu hesaplamanın yapılması gerekmektedir.
#Önceki listemizi getirelim:
df.sort_values("average_count_score", ascending=False).head(20)    

,title,vote_average,vote_count,vote_count_score,average_count_score
15480,Inception,8.10000,14075.00000,10.00000,81.00000
12481,The Dark Knight,8.30000,12269.00000,8.84519,73.41505
22879,Interstellar,8.10000,11187.00000,8.15332,66.04190
17818,The Avengers,7.40000,12000.00000,8.67318,64.18153
14551,Avatar,7.20000,12114.00000,8.74607,62.97174
26564,Deadpool,7.40000,11444.00000,8.31766,61.55065
2843,Fight Club,8.30000,9678.00000,7.18842,59.66388
20051,Django Unchained,7.80000,10297.00000,7.58423,59.15697
23753,Guardians of the Galaxy,7.90000,10014.00000,7.40327,58.48582
292,Pulp Fiction,8.30000,8670.00000,6.54387,54.31414


In [127]:
#Yukarıda bulunan deadpool filminin sırası sanki çok yukarıda bunu bir inceleyelim:
#Bu film için bir skor hesaplaması yapalım:
#Şimdi weighted_rating fonksiyonumuza neyi getirmemiz gerekiyor bu filmin r değeri ne idi? 
#Yukarıda bulunan listemizde average ı ifade ediyor, bunu getireceğiz. İkinci değer ne idi?
#vote_count du bunu getireceğiz. Üçüncü değer ise m ve c değerleri idi.
weighted_rating(7.40000, 11444.00000,M, C)

NameError: name 'M' is not defined

Görüldüğü üzere 7.08 skoru geldi. Daha önceki skor ne idi?
7.40 yukarıda bizim sonucumuzda.

In [126]:
#Örnek-2:
#Inception filmi en tepede listemizde bunu yapalım:
weighted_rating(8.10000, 14075.00000, M, C)

NameError: name 'M' is not defined

Görüldüğü üzere 7.72 skoru geldi.

In [130]:
#Esaretin Bedeli(The Shawshank Redemption) filmi için yapalım:
weighted_rating(8.50000, 8358.00000, M, C)

NameError: name 'M' is not defined

Çıktımızda sonuç 7.83 olarak sonuç verdi.

In [133]:
#Yapmış olduğumuz bu sıralama sonucunda, bu üç değer bizim listemizden farklı dağıldı.
#Esaretin bedeli filmi şuanda bu üç gözlem üzerinden en tepede bulunan film fakat bizim listemizde daha alt taraftaydı.
#O zaman bunu bütün veriye uygulayalım:
#veri setim içersine bir değişken ekliyorum "weighted_rating" adında daha sonra iki sütunu(vote_average ve vote_count) kolayca geçiriyorum.
df["weighted_rating"] = weighted_rating(df["vote_average"],
                                        df["vote_count"], M, C)

NameError: name 'M' is not defined

In [135]:
# Şimdi weighted_rating e göre sıralayalım:
df.sort_values("weighted_rating", ascending = False).head(20)

KeyError: 'weighted_rating'

In [ ]:
Evet bu yukarıda bulunan sıralamanın güven verdiği açıktır.

# Bayes Ortalama Derecelendirme Puanı(Bar Score)

IMDB filmlerini sıralamaya devam edeceğiz.

In [ ]:
IMDB nin 2015 yılına kadar kullandığı formülasyonu kullanarak bir hesaplama işlemi yaptık ve sıraladık. 
Bu sıralama işlemi sonucunda ilk 5 e bakalım:
-The Dark Knight
-The Shawshank Redemption
-Fight Club
-Inception
-Pulp Fiction

In [3]:
#Şimdi biz bar_score yöntemini burada bulunan filmlerin rating lerine göre oluşturmuş olacağız ve birkaç tane film için bu hesaplamayı gerçekleştireceğiz.
def bayesian_average_rating(n, confidence=0.95):        
    if sum(n) ==0:
        return 0
    K =len(n)
    z=st.norm.ppf(1-(1-confidence) / 2)
    N = sum(n)
    first_part = 0.0
    second_part=0.0
    for k, n_k in enumerate(n):
        first_part += (k+1) * (n[k]+1) / (N + K)
        second_part +=(k+1) * (k+1) * (n[k] + 1) / (N +K)
    score = first_part - z * math.sqrt((second_part - first_part * first_part) / (N + K + 1))
    return score

In [7]:
#Esaretin Bedeli için:
bayesian_average_rating([34733, 4355, 4704, 6561,8987, 7865, 4532, 7865, 9087, 7865]) #Elim ile buraya 1 yıldız alan bu kadar vs. 10 yıldıza kadar yazdım.

4.443327558753502

In [8]:
#Baba Filmi için:
bayesian_average_rating([34733, 421355, 477604, 6561,8543987, 786235, 452132, 786543, 9087, 786325]) #Elim ile buraya 1 yıldız alan bu kadar vs. 10 yıldıza kadar yazdım.

5.458533316141464

In [11]:
#Peki bunu bütün filmlere uygulamak istediğimizde vote_count var, average_count var fakat vote_count ların dağılımı yok.
#Bu neden ile bunların olduğu veri setini okutup işlemimize devam edeceğiz:
df= pd.read_csv("C:/Users/emre.kanarya/Desktop/imdb_ratings.csv")
df = df.iloc[0:, 1:] #Problemli bazı satırlardan kurtulmak için bunu yazdık.

In [13]:
df.shape

(250, 13)

Dikkat: Normalde elimizde bulunan bu veri seti içerisinde puan sayısı var, average lar fakat bu puanların dağılımı yoktu. 10 puandan kaç tane, 5 puandan kaç tane vs. 

In [14]:
# Böylelikle artık elimizizde "bayesian_average_rating" yntemini uygulayabileceğimiz formatta bir veri var.

In [15]:
#Sıralamayı yapalım:
df["bar_score"] = df.apply(lambda x: bayesian_average_rating(x[["one","two","three","four","five","six","seven","eight","nine","ten"]]),axis=1)

In [ ]:
#Bar_Score a göre sıralama işlemini görelim:
    

In [20]:
df.sort_values("bar_score", ascending= False).head(20)

,id,movieName,rating,ten,nine,eight,seven,six,five,four,three,two,one,bar_score
0,111161,1. The Shawshank Redemption (1994),9.20000,1295382,600284,273091,87368,26184,13515,6561,4704,4355,34733,9.14539
1,68646,2. The Godfather (1972),9.10000,837932,402527,199440,78541,30016,16603,8419,6268,5879,37128,8.94002
3,468569,4. The Dark Knight (2008),9.00000,1034863,649123,354610,137748,49483,23237,11429,8082,7173,30345,8.89596
2,71562,3. The Godfather: Part II (1974),9.00000,486356,324905,175507,70847,26349,12657,6210,4347,3892,20469,8.81250
4,50083,5. 12 Angry Men (1957),8.90000,246765,225437,133998,48341,15773,6278,2866,1723,1478,8318,8.76793
6,167260,7. The Lord of the Rings: The Return of ...,8.90000,703093,433087,270113,117411,44760,21818,10873,7987,6554,28990,8.75204
5,108052,6. Schindler's List (1993),8.90000,453906,383584,220586,82367,27219,12922,6234,4572,4289,19328,8.74361
11,109830,12. Forrest Gump (1994),8.80000,622104,553654,373644,151284,51140,22720,11692,7647,5941,12110,8.69915
12,1375666,13. Inception (2010),8.70000,724798,627987,408686,174229,60668,26910,13436,8703,6932,17621,8.69315
10,137523,11. Fight Club (1999),8.80000,637087,572654,371752,152295,53059,24755,12648,8606,6948,17435,8.67448
